In this notebook, I focus on pricing European Options using the following models: Black-Scholes-Merton model, Heston model, and the Bates model. 

Starting with the Black-Scholes-Merton model, I find the price of options using analytical formulas. Next, I show how to find the Implied Volatility by observing the market price of the Options. Next, I derive the Option price using Monte Carlo simulations of the Geometric Brownian motion. Then, for the Heston model (which has stochastic volatility), I derive the price of Options using Monte Carlo Simulations. Finally, for the Bates model (which has both jumps and stochastic volatility), I derive the price of Options using Monte Carlo Simulations.      

In [79]:
import math
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import seaborn as sns

Pricing European Call and Put Options using the Black-Scholes-Merton model's analytical pricing formulas

In [80]:
def option_price_bsmd(S, K, T, r, q, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    return price

In [81]:
S = 100      
K = 100   
T = 1        
r = 0.05   
q = 0.02
sigma = 0.2  

In [82]:
call_price = option_price_bsmd(S, K, T, r, q, sigma, 'call')
put_price = option_price_bsmd(S, K, T, r, q, sigma, 'put')

print(f"Call Option Price: {call_price:.2f}")
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 9.23
Put Option Price: 6.33


Finding the implied volatility using the price of Options observed in the market

In [83]:
def implied_volatility(price, S, K, T, r, q, option_type='call', tol=1e-6, max_iter=100):
    def objective(sigma):
        return option_price_bsmd(S, K, T, r, q, sigma, option_type) - price
    imp_vol = brentq(objective, 1e-8, 10.0, xtol=tol, maxiter=max_iter)
    return imp_vol 

In [84]:
call_price_obs = 9.23
put_price_obs = 6.33

In [85]:
iv_call = implied_volatility(call_price_obs, S, K, T, r, q, 'call')
iv_put = implied_volatility(put_price_obs, S, K, T, r, q, 'put')

print(f"Implied Volatility (using market price of Call): {iv_call:.4f}")
print(f"Implied Volatility (using market price of Put): {iv_put:.4f}")

Implied Volatility (using market price of Call): 0.2001
Implied Volatility (using market price of Put): 0.2000


Pricing European Call and Put Options using Monte Carlo simulations of the Geometric Brownian Motion. Presenting the answer using 95% confidence interval.

In [86]:
def bsmd_mc_option_price(S0, K, T, r, q, sigma, n_sim=10**6, option_type='call'):
    Z = np.random.standard_normal(n_sim)
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    if option_type == 'call':
        payoffs = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - ST, 0)
    discounted_payoffs = np.exp(-r * T) * payoffs
    option_price = np.mean(discounted_payoffs)
    std_err = np.std(discounted_payoffs, ddof=1) / np.sqrt(n_sim)
    return option_price, std_err

In [87]:
S0 = 100      
K = 100       
T = 1         
r = 0.05     
q = 0.02
sigma = 0.2   
zs=1.96

In [89]:
call_price, se_call = bsmd_mc_option_price(S0, K, T, r, q, sigma, option_type='call')
put_price, se_put  = bsmd_mc_option_price(S0, K, T, r, q, sigma, option_type='put')

print(f"Call Option Price: {call_price:.4f} ± {zs*se_call:.4f}")
print(f"Put Option Price:  {put_price:.4f} ± {zs*se_put:.4f}")

Call Option Price: 9.2246 ± 0.0271
Put Option Price:  6.3550 ± 0.0180


Pricing European Call and Put Options using Monte Carlo simulations of the **Heston Model** 

The model is defined by: $dS=(r-q)~dt+\sqrt{v} ~dW_S$ and $dv=\kappa(\theta-v)~dt + \xi ~\sqrt{v}~dW_v $ with $\text{corr}(dW_S,dW_v)=\rho$

In [90]:
def heston_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, n_sim=10**6, n_steps=252, option_type='call'):
    dt = T / n_steps
    S = np.full((n_sim,), S0, dtype=np.float64)
    v = np.full((n_sim,), v0, dtype=np.float64)
    for _ in range(n_steps): 
        v = np.maximum(v, 0) 
        W1 = np.random.standard_normal(size=n_sim)
        W2 = rho * W1 + np.sqrt(1 - rho**2) * np.random.standard_normal(size=n_sim)
        S = S * np.exp((r - q - 0.5 * v) * dt + np.sqrt(v * dt) * W1)
        v = v + kappa * (theta - v) * dt + xi * np.sqrt(v * dt) * W2
    if option_type == 'call':
        payoffs = np.maximum(S - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - S, 0)
    discounted_payoffs = np.exp(-r * T) * payoffs
    option_price = np.mean(discounted_payoffs)
    std_err = np.std(discounted_payoffs, ddof=1) / np.sqrt(n_sim)
    return option_price, std_err

In [91]:
S0 = 100
K = 100
T = 1
r = 0.05
q=0.0
zs = 1.96

v0 = 0.04        
kappa = 2.0       
theta = 0.04      
xi = 0.4         
rho = 0.9       

In [92]:
call_price, se_call = heston_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, option_type='call')
put_price, se_put = heston_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, option_type='put')

print(f"Call Option Price: {call_price:.4f} ± {zs*se_call:.4f}")
print(f"Put Option Price: {put_price:.4f} ± {zs*se_put:.4f}")

Call Option Price: 9.6962 ± 0.0387
Put Option Price: 4.8279 ± 0.0117


Pricing European Call and Put Options using Monte Carlo simulations of the **Bates Model**.

The model is defined by: $dS=(r-q)~dt+\sqrt{v} ~dW_S + dZ_p$ and $dv=\kappa(\theta-v)~dt + \xi ~\sqrt{v}~dW_v $ with $\text{corr}(dW_S,dW_v)=\rho$. 

$dZ_p$ is a compound Poisson process with intensity $\lambda_j$ and random jump sizes $k$. 

$k$ is distributed as $\ln(1+k)\sim N(\mu_j,\sigma_j^2)$ where $\mu_j=\ln(1+\bar k)-\frac{\sigma_j^2}{2}$ and $\bar k$ is the mean jump size.

In [105]:
def bates_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, lambda_j, mu_j, sigma_j, n_sim=10**6, n_steps=252, option_type='call'):
    dt = T / n_steps
    S = np.full(n_sim, S0)
    v = np.full(n_sim, v0)
    kappa_jump = np.exp(mu_j + 0.5 * sigma_j**2) - 1  
    for _ in range(n_steps):
        v = np.maximum(v, 0)
        W1 = np.random.standard_normal(size=n_sim)
        W2 = rho * W1 + np.sqrt(1 - rho**2) * np.random.standard_normal(size=n_sim)  
        dN = np.random.poisson(lambda_j * dt, size=n_sim)  
        jump_sum = dN * np.random.normal(loc=mu_j, scale=sigma_j, size=n_sim)
        S = S * np.exp((r - q - lambda_j * kappa_jump - 0.5 * v) * dt +
                       np.sqrt(v * dt) * W1 + jump_sum)
        v = v + kappa * (theta - v) * dt + xi * np.sqrt(v * dt) * W2
    if option_type == 'call':
        payoffs = np.maximum(S - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - S, 0)
    discounted_payoffs = np.exp(-r * T) * payoffs
    option_price = np.mean(discounted_payoffs)
    std_err = np.std(discounted_payoffs, ddof=1) / np.sqrt(n_sim)
    return option_price, std_err

In [106]:
S0 = 80
K = 76
T = 0.5
r = 0.03
q = 0.02

v0 = 0.04
kappa = 1.0
theta = 0.05
xi = 0.2
rho = -0.7

lambda_j = 2.0   
sigma_j = 0.08
k_bar = 0.02
mu_j = np.log(1+k_bar)- (sigma_j**2)/2   

In [107]:
call_price, se_call = bates_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, lambda_j, mu_j, sigma_j, option_type='call')
put_price, se_put = bates_mc_option_price(S0, K, T, r, q, v0, kappa, theta, xi, rho, lambda_j, mu_j, sigma_j, option_type='put')

print(f"Call Option Price: {call_price:.4f} ± {zs*se_call:.4f}")
print(f"Put Option Price: {put_price:.4f} ± {zs*se_put:.4f}")

Call Option Price: 7.5651 ± 0.0184
Put Option Price: 3.2332 ± 0.0115
